In [ ]:
from datetime import date
from datetime import date
from datetime import datetime, timedelta
import xalpha as xa
import pandas as pd
import numpy as np

In [ ]:
"""获取年度涨幅"""
def get_year_rate(code):
    last_trade_day=datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d')
    first_day_of_year='2020-01-02'
    fake_buy = 10000 #假设买入10000元
    b_jz = get_jz(code,first_day_of_year)
    e_jz = get_jz(code,last_trade_day)

    buy_fe = fake_buy/b_jz

    current_money = e_jz * buy_fe

    fundinfo = xa.fundinfo(code)
    #考虑分红情况
    fenghong = get_fenhong(code)
    fenghong_zonge = fenghong * buy_fe
    add = ( fenghong_zonge + current_money - fake_buy)/fake_buy
    # print('{},{},add={}',b_jz,e_jz,add)

    return add

import numpy as np
#用于计算基金的年度涨幅
def get_fenhong(code):
    fundinfo = xa.fundinfo(code)
    # print(fundinfo.fenhongdate)
    if(fundinfo.fenhongdate):
        # print(fundinfo.special.loc[])
        s = fundinfo.special
        # print(s.dtypes)
        # print(s[s['date']])
        tmp = s[s.date > np.datetime64('2020-01-01 00:00:00')]
        # print(tmp)

        comment = tmp['comment']
        fenhong = comment.sum()
        # print('分红总额为{}'.format(fenhong))
        for _,value in comment.items():
            print(value)

        return fenhong
    else:
        # print('分红总额为{}'.format(0))
        return 0



In [ ]:

def get_jz(code,date):
    fundinfo = xa.fundinfo(code)
    df=fundinfo.price
    # print(df)
    
    #获取某一日期净值 如果没有则返回最后一个交易日净值
    if df[df['date']== date].empty:
        dwjz = df.tail(1)['netvalue']
    else:
        dwjz = df[df['date']==date]['netvalue'] 

    for index in dwjz.index:
        # print('{},date:{}单位净值为:{}'.format(fundinfo.name,date,dwjz.get(index)))
        return dwjz.get(index)

#查询某一基金截止某一日期的份额,用于计算分红
def get_fener(deadline_date,code):
    read=xa.record("./tests/fund_2020.csv",skiprows=1)
    trade_info = read.status.loc[:,['date',code]]
    trade_info = trade_info[trade_info.date < np.datetime64(deadline_date)]
    # print(trade_info)

    fundinfo = xa.fundinfo(code)

    #计算份额变动
    fe = 0
    for _, row in trade_info.iterrows():
        date = row.get('date')
        dwjz = get_jz(code,date)

        v = row.get(code) #买入或者卖出金额
        if v > 0:
            # print('申购费率:{}'.format(fundinfo.rate)) 
            fe += v*(1-fundinfo.rate/100.)/dwjz
        else:
            fe += v/dwjz #这里没有考虑赎回的费率 注意这里不是减法　卖出的话v为负值

    print('截止{}持有份额{}'.format(deadline_date,fe))
    return fe

def cal_fenhong(code):
    fundinfo = xa.fundinfo(code)

    total_fenhong = 0
    # print(fundinfo.fenhongdate)
    if(fundinfo.fenhongdate):
        s = fundinfo.special
        dates = s[s.date > np.datetime64('2020-01-01 00:00:00')]['date'] #
        fenhongs = s[s.date > np.datetime64('2020-01-01 00:00:00')]['comment']
        # print(tmp.date)

        for index,date in dates.items():
            # print(type(date))
            # print(str(date)[:10])
            fener = get_fener(date,code) #计算分红日持有份额
            fenhong_per_fener = fenhongs[index] #分红日每一份额分红金额
            
            total_fenhong += fenhong_per_fener * fener
    
    print('总计分红{}'.format(total_fenhong))
    return total_fenhong

def f(trade_info,code): #某个基金的交易信息
    fundinfo = xa.fundinfo(code)

    buy,sell=0,0
    fe=0
    for index, row in trade_info.iterrows():
        date = row.get('date')
        dwjz = get_jz(code,date)

        v = row.get(code)
        if v > 0:
            # print('申购费率:{}'.format(fundinfo.rate)) 
            buy += v
            fe += v*(1-fundinfo.rate/100.)/dwjz #份额变动 要考虑申购及赎回费率 
        else:
            sell += -v
            fe += v*(1-fundinfo.rate/100.)/dwjz #这里没有做赎回费率的处理 v是负数
        
    last_trade_day=datetime.strftime(datetime.now() - timedelta(1), '%Y-%m-%d')
    dqjz = get_jz(code,last_trade_day) #api接口到24点以后才更新净值,所以做多只能取到昨日净值
    # print(dqjz)
    
    fenhong = cal_fenhong(code)
    dqye = fe*dqjz+sell+fenhong #包括当前持有金额以及已卖出金额之和
    syl = dqye/buy-1#收益率
    print('{},{}--份额:{},昨日净值{},当前余额{},总投入{},收益率{},盈利{}'.format(fundinfo.name,last_trade_day,fe,dqjz,dqye,buy,syl,dqye-buy))
    # print('{},当前余额{:.2f},总投入{:.2f},收益率{:.3f},盈利{:.2f}'.format(fundinfo.name,dqye,buy,syl,dqye-buy))
    return (fe,dqjz,dqye,buy,syl)


def trade_analysis():
    read=xa.record("./tests/fund_2020.csv",skiprows=1)
    df_list = []
    jjcode_list = list(read.status.columns.values)[1:] #持有的全部基金列表
    t_buy,t_get = 0,0 #总计买入,总计剩余(包括当前剩余及赎回)
    for code in jjcode_list:
        trade_info = read.status.loc[:,['date',code]]
        # print(trade_info)

        fe,dqjz,dqye,buy,syl = f(trade_info,code)
        t_buy += buy
        t_get += dqye

        fundinfo = xa.fundinfo(code)
        year_add = get_year_rate(code) #年初买入持有不动的收益率
        df_list.append([fundinfo.name,dqye,buy,dqye-buy,(dqye-buy)/buy,year_add])
    # print('总投入:{},总回报:{},整体收益率:{:.3f},盈利{}\n'.format(t_buy,t_get,t_get/t_buy-1,t_get-t_buy))

    return df_list




In [ ]:
lst = trade_analysis()
df = pd.DataFrame(lst,columns =['Name','当前余额','总投入','收益','收益率','当年涨幅'])

In [ ]:
df

In [ ]:
fuck = df
total_yuer = df['当前余额'].sum()
total_buy = df['总投入'].sum()
total_add = df['收益'].sum()
df.loc[9,:]=['总额',total_yuer,total_buy,total_add,total_add/total_buy,df['当年涨幅'].mean()]
# fuck.loc['总额',:] = fuck.apply(lambda x:x.sum())

In [ ]:
df

In [ ]:
# df['当前余额']=df['当前余额'].map('{:,.0f}'.format)
# df['收益率'] = df['收益率'].map('{:,.3f}'.format)
# df['收益'] = df['收益'].map('{:,.0f}'.format)
df['当前余额']=df['当前余额'].round(decimals=0)
df['收益']=df['收益'].round(decimals=0)
df['收益率']=df['收益率'].round(decimals=4)
df['占比']=df['当前余额']/total_yuer

In [ ]:
df.sort_values(by=['总投入'],ascending=False)

In [ ]:
#df['当前余额'] = pd.to_numeric(df['当前余额']) 

In [ ]:
df['当前余额'].sum()

In [ ]:
# new_df=df.iloc[1:,:]
# new_df.loc[:,'当前余额'].sum()